In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint
from math import pi, sqrt, log, degrees
import numpy as np
import scipy.sparse.linalg as linalg
import matplotlib

from matplotlib import cm

pd.set_option('display.precision',14)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 15)
pd.set_option('display.width', 1000)
pd.set_option('display.width', 1000)

%matplotlib
FONTSIZE=22

Using matplotlib backend: TkAgg


In [26]:
def phi_psi_to_file(path):

    import glob
    import os
    import matplotlib.pyplot as plt
    from math import pi, radians

    micro_states = glob.glob('%s/new_run*.gro'%path)
    print 'Found {0} .gro microstates'.format(len(micro_states))
    
    #macro_states = glob.glob('%s/new_run*.gro'%path)
    #print 'Found {0} .gro microstates'.format(len(macro_states))
    
    pdb = glob.glob('%s/maxstate*.pdb'%path)
    
    f = open('phi_psi.txt','w')
    f.close()
    
    for conf in micro_states:

        os.system('gmx chi -f {0} -s {1} -rama'.format(conf, pdb[0]))
        f=open('ramaPhiPsiALA2.xvg','r')
        lines = f.readlines()
        f.close()
        p1 = float(lines[-1].strip().split(' ')[0].strip())
        p2 = float(lines[-1].strip().split(' ')[-1].strip())
        phi = p1
        psi = p2
        k = os.path.basename(conf)
        os.remove('ramaPhiPsiALA2.xvg')
        os.remove('chi.log')
        os.remove('order.xvg')
        f = open('phi_psi.txt','a')
        line = str(k.split('.')[0].split('_')[2]) + ',' + str(phi) + ',' + str(psi) + '\n'
        f.write(line)

In [29]:
def get_prob(path):
    
    assigns = np.load('%s/assignment.npy'%path)
    map_assigns = np.load('%s/mapped_assignments.npy'%path)

    
    probs = []
    read_mat = np.load('%s/macrostate_tx_mat.npy'%path)
    csr_mat = read_mat.item()
    dense_mat = csr_mat.todense()
    
    evalue, evector = linalg.eigs(dense_mat,k=8)
    evalue, evector = evalue.real, evector.real
    
    cnt=0
    for val in evalue:
        if val == evalue.max():
            break
        cnt+=1
            
    ev = abs(evector[:,cnt])

    #for item in ev:
    #    val = np.asscalar(item.real)
    #    #print val
    #    probs.append(-1*log(val))
    
    data = dict()
    
    for (x,y), value in np.ndenumerate(map_assigns):
        
        macro_state = map_assigns[x][y]
        if macro_state not in data.keys():
            data[macro_state] = {'microstates': [], 'prob': 0}
            
        micro_state = assigns[x][y]
        
        if micro_state not in data[macro_state]['microstates']:
            data[macro_state]['microstates'].append(micro_state)
        
    for i in range(len(ev)):
        
        if i in data.keys():
            data[i]['prob'] = ev[i].real
        
    print data
    return data

In [35]:
def pmf_plot(path, states_prob):
    
    f = open('phi_psi.txt','r')
    lines = f.readlines()
    phi_psi=[]
    
    for val in lines:
        microstate = int(val.strip().split(',')[0].strip())
        phi = float(val.strip().split(',')[1].strip())
        psi = float(val.strip().split(',')[2].strip())
        
        for key, val in states_prob.iteritems():
            
            if microstate in val['microstates']:
                prob = val['prob']/len(val['microstates'])
                print 'macro-prob: ', val['prob'], 'micro-prob: ',prob
                break
                
        val = np.asscalar(prob)
        fe = -1*log(val)
        
        phi_psi.append((phi, psi, fe))
    
    f.close() 
    
    print 'Len: ', len(phi_psi)
    print 'Len uniq: ', len(set(phi_psi))
    print phi_psi
    plot(phi_psi,path)

In [39]:
def plot(phi_psi, path):
    plt.figure()
    
    empty_mat = np.zeros((20,20),dtype=float)
    #extent1 = [min(phi), max(phi), min(psi), max(psi)]
    extent2 = [-180, 180,-180, 180]
    
    #phi_linspace = np.linspace(-180,180, num=36)
    #psi_linspace = np.linspace(180,-180, num=36)
    
    #print phi_linspace
    #print psi_linspace
    
    phi_linspace = range(-180,190,18)
    psi_linspace = range(180,-190,-18)

    #phi_coords=[]
    #psi_coords=[]
    cnt=0
    phi_vals = []
    psi_vals = []
    fe_vals = []
    for cnt in range(len(phi_psi)):
        #print '------------------------------'
        #print phi[cnt]
        #print psi[cnt]
        
        for i in range(0,len(phi_linspace)-1):
            if float(phi_linspace[i]) < float(phi_psi[cnt][0]) < float(phi_linspace[i+1]):
                #print float(phi_linspace[i]), float(phi[cnt]), float(phi_linspace[i+1])
                #print i
                y=i
                break
        
        for j in range(0,len(psi_linspace)-1):
            if float(psi_linspace[j]) > float(phi_psi[cnt][1]) > float(psi_linspace[j+1]):
                #print float(psi_linspace[i]), float(psi[cnt]), float(psi_linspace[i+1])
                #print j
                x=j
                break
        
        #y = min(range(len(phi_linspace)), key=lambda i: abs(phi_linspace[i]-phi[cnt]))
        #x = min(range(len(psi_linspace)), key=lambda i: abs(psi_linspace[i]-psi[cnt]))
                
        #print 'phi: ',y,'psi: ',x
        #print val
        #print cnt
        #print '------------------------------'
        empty_mat[x,y] += float(phi_psi[cnt][2])
        #phi_coords.append(y)
        #psi_coords.append(x)
        phi_vals.append(float(phi_psi[cnt][0]))
        psi_vals.append(float(phi_psi[cnt][1]))
        fe_vals.append(float(phi_psi[cnt][2]))

    #plt.clf()
    for i in range(len(empty_mat)):
        for j in range(len(empty_mat)):
            if float(empty_mat[i,j]) == 0.0:
                empty_mat[i,j] = empty_mat.max()
    ax = plt.imshow(empty_mat, extent=extent2, cmap = cm.get_cmap('jet_r'), interpolation = "gaussian")

    colors = []
    #colorm = cm.get_cmap('jet_r')
    #for i in range(len(phi)):
        #print phi[i], psi[i]
        #c = colorm(empty_mat[phi_coords[i]][psi_coords[i]])
        #print c
        #colors.append(c)
    
    plt.title('Free Energy Plot - %s'%path, fontsize=FONTSIZE)
    plt.xlabel(r'$\Phi$ Angle [degrees]', fontsize=FONTSIZE)
    plt.ylabel(r'$\Psi$ Angle [degrees]', fontsize=FONTSIZE)
    
    xticks_1 = np.arange(-180,180, 18)
    xticks = np.append(xticks_1,180)
    yticks_1 = np.arange(-180, 180,18)
    yticks = np.append(yticks_1,180)
    plt.xticks(xticks,rotation=90, fontsize=FONTSIZE)
    plt.yticks(yticks, fontsize=FONTSIZE)
    
    cb = plt.colorbar(pad=0.04)
    cb.set_label('Free Energy',size=FONTSIZE)
    for t in cb.ax.get_yticklabels():
         t.set_fontsize(FONTSIZE) 
    #plt.scatter(phi,psi)
    
    plt.grid()
    plt.show()
    
    fig = plt.gcf()
    fig.set_size_inches(16, 16)
    fig.savefig('plots/free_energy_%s.png'%path, dpi=100)
    

In [41]:
import glob

for i in range(1,11):
    
    states_prob = get_prob('iter%s'%i)
    phi_psi_to_file('iter%s'%i)
    pmf_plot('iter%s'%i, states_prob)

{0: {'microstates': [58, 1, 55, 86, 83, 20, 30], 'prob': 0.31622776601689601}, 1: {'microstates': [0, 97, 80, 46, 34, 91, 56, 31, 73, 85, 62, 96, 12, 26, 67, 44, 36, 15], 'prob': 0.31622776601676783}, 2: {'microstates': [24, 65, 43, 18, 79, 99, 45, 70, 93, 2, 82, 14, 84, 87], 'prob': 0.31622776601676694}, 3: {'microstates': [78, 29, 52, 81, 3, 76, 49, 16, 59], 'prob': 0.31622776601687763}, 4: {'microstates': [41, 13, 60, 37, 7, 23, 63, 75, 38, 72, 19, 54, 27, 71, 61, 4], 'prob': 0.31622776601689828}, 5: {'microstates': [40, 90, 6, 69, 22, 92], 'prob': 0.3162277660168975}, 6: {'microstates': [33, 17, 89, 98, 35, 39, 11, 74, 48, 66, 21, 42, 64, 77], 'prob': 0.3162277660167685}, 7: {'microstates': [9, 57], 'prob': 0.31622776601684}, 8: {'microstates': [68, 53, 95, 5, 47, 32, 94, 8, 51], 'prob': 0.31622776601676861}, 9: {'microstates': [28, 50, 25, 10, 88], 'prob': 0.31622776601689784}}
Found 100 .gro microstates
macro-prob:  0.316227766017 micro-prob:  0.0351364184463
macro-prob:  0.31622